## Step 3:

Import your analysis libraries, as well as your new customer data set as a dataframe.

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [5]:
path = r'C:\Users\Jenna\Desktop\Data Course\Part4_Instacart_analysis'

In [6]:
df_customers = pd.read_csv(os.path.join(path, '02_Data', 'Original_Data','customers.csv'), index_col = False)

## Step 4:

Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [7]:
# Step 1: Check the shape of the dataframe

df_customers.shape

(206209, 10)

In [8]:
# Step 2: Check the header to see column names

df_customers.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

###  4a.  The names of the customers are probably not relevant to this specific analysis but I will not drop them for now

### 4b. I will rename sunam to 'Last Name' to stay consistent with the First name column, and rename n-dependents to number_of_dependents for clarity

In [9]:
# rename surname column

df_customers.rename(columns = {'Surnam': 'Last Name'}, inplace = True)

In [10]:
# rename n_dependents column

df_customers.rename(columns = {'n_dependants': 'Number_of_Dependants'}, inplace = True)

In [11]:
# check the output

df_customers.head()

user_id First Name Last Name  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   Number_of_Dependants fam_status  income  
0                     3    married  165665  
1                     0     single   59285  
2                     2    married   99568  
3                     0     single   42049  
4                     1    married   40374

## Step 5:

Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data.

### 5a. Check the data types

In [12]:
# Check data types

df_customers.dtypes

user_id                  int64
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number_of_Dependants     int64
fam_status              object
income                   int64
dtype: object

### 5a. We can recategorize user_id as string since it is not quantitative, the others should be ok

In [13]:
# change userid from int to string

df_customers['user_id'] = df_customers['user_id'].astype('str')

In [14]:
# check that the change worked

print(df_customers['user_id'].dtypes)

object


### 5b. The describe function can give us basic idea of the data

In [15]:
df_customers.describe()

Age  Number_of_Dependants         income
count  206209.000000         206209.000000  206209.000000
mean       49.501646              1.499823   94632.852548
std        18.480962              1.118433   42473.786988
min        18.000000              0.000000   25903.000000
25%        33.000000              0.000000   59874.000000
50%        49.000000              1.000000   93547.000000
75%        66.000000              3.000000  124244.000000
max        81.000000              3.000000  593901.000000

### 5b. It is a little suprising that the max dependents is 3, but overall there are no values in here that seem unreasonable

### 5c. Check for missing values

In [16]:
# check the data frame for null values in each column

df_customers.isnull().sum()

user_id                     0
First Name              11259
Last Name                   0
Gender                      0
STATE                       0
Age                         0
date_joined                 0
Number_of_Dependants        0
fam_status                  0
income                      0
dtype: int64

### 5c. There are quite a few missing first names, but this is not an important data point so I will not make any changes

### 5d. Check for mixed data types

In [17]:
# check for columns that have mixed data types that may make analysis in the future complicated

for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_customers[weird]) > 0:
        print (col)

First Name


### 5d. The first name column may be considered mixed since it contains NaN values, we can assign the data type as string

In [18]:
# assign First Name as string

df_customers['First Name'] = df_customers['First Name'].astype('str')

In [19]:
# Rerun the mixed data check to see if this worked

for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_customers[weird]) > 0:
        print (col)

### 5d. There are no longer and mixed data type columns

### 5e. Check for duplicates

In [20]:
# make a new dataframe that consists of duplciated entries

df_dups = df_customers[df_customers.duplicated()]

In [21]:
# check the number of duplicate entries by looking at this new dataframe

df_dups.shape

(0, 10)

### 5e. There are no duplicates in this data frame

## Step 6:

Combine your customer data with the rest of your prepared Instacart data. (Hint: Make sure the key columns are the same data type!)

### 6a. Import the running dataframe to combine with customers df

In [22]:
df_merged = pd.read_pickle(os.path.join(path, '02_Data', 'Prepared_Data','merged_exercise8_update.pkl'))

In [23]:
# Check the head to see what key will be used

df_merged.head()

order_id  user_id  user_order_number  orders_day_of_week  \
0   2539329        1                  1                   2   
1   2398795        1                  2                   3   
2    473747        1                  3                   3   
3   2254736        1                  4                   4   
4    431534        1                  5                   4   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                  8                     NaN         196                  1   
1                  7                    15.0         196                  1   
2                 12                    21.0         196                  1   
3                  7                    29.0         196                  1   
4                 15                    28.0         196                  1   

   reordered product_name  ...    price_range_loc     busiest_day  \
0          0         Soda  ...  Mid range product  Regularly Busy   
1          1         Soda  ...  Mid range product  Regularly Busy   
2          1         Soda  ...  Mid range product  Regularly Busy   
3          1         Soda  ...  Mid range product      Least Busy   
4          1         Soda  ...  Mid range product      Least Busy   

     busiest_days busiest_period_of_day max_order  loyalty_flag average_order  \
0  Regularly Busy        Average Orders        10  New Customer      6.367797   
1    Slowest days        Average Orders        10  New Customer      6.367797   
2    Slowest days           Most Orders        10  New Customer      6.367797   
3    Slowest days        Average Orders        10  New Customer      6.367797   
4    Slowest days           Most Orders        10  New Customer      6.367797   

   spending_flag order_freq              freq_flag  
0    Low Spender       20.5  Non-frequent Customer  
1    Low Spender       20.5  Non-frequent Customer  
2    Low Spender       20.5  Non-frequent Customer  
3    Low Spender       20.5  Non-frequent Customer  
4    Low Spender       20.5  Non-frequent Customer  

[5 rows x 23 columns]

### 6b. The data frames will be merged on user_id

In [24]:
# make sure that the user_id in our dataframe is also a string

df_merged['user_id'] = df_merged['user_id'].astype('str')


In [ ]:
# merged the data sets on user id

df_combined = df_merged.merge(df_customers, on = 'user_id', indicator = 'True')

In [28]:
# check the new dataframe

df_combined.head()

order_id user_id  user_order_number  orders_day_of_week  order_hour_of_day  \
0   2539329       1                  1                   2                  8   
1   2398795       1                  2                   3                  7   
2    473747       1                  3                   3                 12   
3   2254736       1                  4                   4                  7   
4    431534       1                  5                   4                 15   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN         196                  1          0   
1                    15.0         196                  1          1   
2                    21.0         196                  1          1   
3                    29.0         196                  1          1   
4                    28.0         196                  1          1   

  product_name  ...  First Name  Last Name  Gender    STATE Age date_joined  \
0         Soda  ...       Linda     Nguyen  Female  Alabama  31   2/17/2019   
1         Soda  ...       Linda     Nguyen  Female  Alabama  31   2/17/2019   
2         Soda  ...       Linda     Nguyen  Female  Alabama  31   2/17/2019   
3         Soda  ...       Linda     Nguyen  Female  Alabama  31   2/17/2019   
4         Soda  ...       Linda     Nguyen  Female  Alabama  31   2/17/2019   

  Number_of_Dependants  fam_status income  True  
0                    3     married  40423  both  
1                    3     married  40423  both  
2                    3     married  40423  both  
3                    3     married  40423  both  
4                    3     married  40423  both  

[5 rows x 33 columns]

## Step 8.

Export this new dataframe as a pickle file so you can continue to use it in the second part of this task.Export this new dataframe as a pickle file so you can continue to use it in the second part of this task.

In [ ]:
df_combined.to_pickle(os.path.join(path, '02_Data', 'Prepared_Data', 'customer_merged.pkl'))